In [1]:
import array
import random
import time
import numpy as np
from scipy.stats import rankdata as rd
from itertools import combinations
from collections import Counter
from tqdm import tqdm

### Import HandRanks lookup tables

In [2]:
# Import two plus two HandRanks dat file from
# https://github.com/christophschmalhofer/poker/blob/master/XPokerEval/XPokerEval.TwoPlusTwo/HandRanks.dat
s = time.time()
handsDB = open('HandRanks.dat', 'rb')
ranks = array.array('i') #signed integer
ranks.fromfile(handsDB, 32487834)
handsDB.close()
e = time.time()
print ("fromfile took", e-s)

# write file to disk for availability in Julia
# with open('rks','w') as outfile:
#     outfile.write(repr(ranks))

fromfile took 0.0827779769897461


### Definitions

In [3]:
TT_to_CactKev = {9: 7452, # 10 straight flushes
                 8: 7296, # 156 four of a kinds
                 7: 7140, # 156 full houses
                 6: 5863, # 1277 flushes
                 5: 5853, # 10 straights
                 4: 4995, # 858 three of a kinds
                 3: 4137, # 858 two pairs
                 2: 1277, # 2860 pairs
                 1: 0}    # 1277 high cards

suits = {'c':'\u2663',
         'd':'\u2662',
         'h':'\u2661',
         's':'\u2660'}

def choose(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n-k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0

def get22Rank(hand):
# https://www.reddit.com/r/learnpython/comments/6dqv39/comment/di4totk/
    p = 53
    for card in hand:
        p = ranks[p+card]
    return p

def getCKEquiv(hand):
    rank = get22Rank(hand)
    handClass = rank >> 12
    rankInClass = rank & 0x00000FFF
    equivRank = 7463-TT_to_CactKev[handClass]-rankInClass
    return equivRank

def getHandClass(rank):
    if rank <=   10: return "Straight flush"
    if rank <=  166: return "Four of a kind"
    if rank <=  322: return "Full house"
    if rank <= 1599: return "Flush"
    if rank <= 1609: return "Straight"
    if rank <= 2467: return "Three of a kind"
    if rank <= 3325: return "Two pair"
    if rank <= 6185: return "Pair"
    else: return "High card"    

def get_pretty_card(card_int):
    return deck_int_to_str[card_int][0]+suits[deck_int_to_str[card_int][1]]

deck_int_to_str = dict(zip(range(1,53), [i+j for i in "23456789TJQKA" for j in "cdhs"]))
deck_str_to_int = {v: k for k, v in deck_int_to_str.items()}
deck = list(deck_int_to_str)

hand_classes = ['Straight flush', 'Four of a kind', 'Full house', 'Flush', 
                'Straight', 'Three of a kind', 'Two pair', 'Pair', 'High card']

getHandClassVec = np.vectorize(getHandClass)

### Deal hole cards, calculate pre-flop equity and potential hand distribution

In [4]:
%%time
n_players = 6
n_sims = min(choose(52-2*n_players,5), 10**5)
count_classes = False

# Deal hole cards, get pretty suits
#hole_cards = [deck_str_to_int[x] for x in ['Ac','Kd','7d','2d','5c','Js','Ad','Kc','7h','2s','5h','Jd']]
hole_cards = random.sample(deck, k=n_players*2)
holes_by_player = [[hole_cards[i], hole_cards[i+n_players]] for i in range(n_players)]
hole_cards_pretty = [get_pretty_card(x) for x in hole_cards]
holes_by_player_pretty = [[hole_cards_pretty[i], hole_cards_pretty[i+n_players]] for i in range(n_players)]

# Compute and print pre-flop equity using Monte Carlo sim
rank_array = np.zeros([n_sims, n_players])
avail_for_flop = [x for x in deck if x not in hole_cards]

if n_sims < choose(52-2*n_players,5):
    community_card_samples = random.sample(list(combinations(avail_for_flop, 5)), n_sims)
else:
    community_card_samples = list(combinations(avail_for_flop, 5))

t0 = time.time()
for k, community_cards in enumerate(tqdm(community_card_samples)):
    for i in range(n_players):
        rank_array[k, i] = getCKEquiv([*holes_by_player[i], *community_cards])
t1 = time.time()
print("{:,.0f}".format(n_sims*n_players/(t1-t0)), "hands per second\n")

# use argmin here as scipy rankdata slow for large n_sims
win_counter = Counter(rank_array.argmin(axis=1))
preflop_equity = [win_counter[i]/n_sims for i in range(n_players)]

print("Pre-flop equity:")
for j in range(n_players):
    print("{:5.0%}".format(preflop_equity[j]), "",
          *holes_by_player_pretty[j])
    
# Get counts of hand by class
if count_classes:
    counts = Counter(getHandClassVec(rank_array).reshape(1,-1)[0])
    print("")
    for c in hand_classes:
        print("{:7.2%}".format(counts[c]/n_sims/n_players), c)
    print("")

100%|██████████| 100000/100000 [00:01<00:00, 72709.70it/s]


434,369 hands per second

Pre-flop equity:
  13%  7♡ 4♡
  10%  6♡ 7♣
  15%  6♠ J♣
  21%  A♡ 8♡
  34%  T♢ A♢
   8%  5♣ J♢
Wall time: 1.61 s


### Flop

In [10]:
# Will be exact equity as # of combos small enough to not have to use Monte Carlo
combos_post_flop = choose(52-2*n_players-3, 2)
rank_array = np.zeros([combos_post_flop, n_players])

flop = random.sample(avail_for_flop, k=3)
pretty_flop = [get_pretty_card(x) for x in flop]
avail_for_turn = [x for x in avail_for_flop if x not in flop]

print("Flop:", *pretty_flop)

for k, turn_and_river in enumerate(combinations(avail_for_turn, 2)):
    for i in range(n_players):
        rank_array[k, i] = getCKEquiv([*holes_by_player[i], *flop, *turn_and_river])

wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
postflop_equity = np.sum(wins_array/wins_total, axis=0)/combos_post_flop
        
print("\nPost-flop equity and improvement vs. pre-flop:")
for j in range(n_players):
    print("{:5.0%}".format(postflop_equity[j]), 
          "{:+5.0%}".format(postflop_equity[j]-preflop_equity[j]), " ",
          *holes_by_player_pretty[j])

Flop: 4♠ K♡ K♣

Post-flop equity and improvement vs. pre-flop:
  32%  +19%   7♡ 4♡
   9%   -0%   6♡ 7♣
  11%   -4%   6♠ J♣
  16%   -4%   A♡ 8♡
  23%  -11%   T♢ A♢
   9%   +1%   5♣ J♢


### Turn

In [11]:
# Will be exact equity as # of combos small enough to not have to use Monte Carlo
combos_post_turn = 52-2*n_players-3-1
rank_array = np.zeros([combos_post_turn, n_players])

turn = random.sample(avail_for_turn, k=1)
pretty_turn = get_pretty_card(turn[0])
avail_for_river = [x for x in avail_for_turn if x not in turn]

print("Flop:", *pretty_flop)
print("Turn:", pretty_turn)

for k, river in enumerate(combinations(avail_for_river, 1)):
    for i in range(n_players):
        rank_array[k, i] = getCKEquiv([*holes_by_player[i], *flop, turn[0], river[0]])

wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
postturn_equity = np.sum(wins_array/wins_total, axis=0)/combos_post_turn

print("\nPost-turn equity and improvement vs. flop:")
for j in range(n_players):
    print("{:5.0%}".format(postturn_equity[j]), 
          "{:+5.0%}".format(postturn_equity[j]-postflop_equity[j]), "",
          *holes_by_player_pretty[j])

Flop: 4♠ K♡ K♣
Turn: 6♣

Post-turn equity and improvement vs. flop:
   8%  -24%  7♡ 4♡
  12%   +3%  6♡ 7♣
  54%  +43%  6♠ J♣
   8%   -8%  A♡ 8♡
  14%   -9%  T♢ A♢
   3%   -6%  5♣ J♢


### River

In [12]:
rank_array = np.zeros([1, n_players])

river = random.sample(avail_for_river, k=1)
pretty_river = get_pretty_card(river[0])

print("Flop:", *pretty_flop)
print("Turn:", pretty_turn)
print("River:", pretty_river)

winning_hand = 7463
for i in range(n_players):
    rank_array[0, i] = getCKEquiv([*holes_by_player[i], *flop, turn[0], river[0]])
    
print("\nFinal hand rankings and improvement vs. turn:")
for j in range(n_players):
    print("{:5.0%}".format(np.sum(rank_array.argmin(axis=1)==j)), 
          "{:+5.0%}".format(np.sum(rank_array.argmin(axis=1)==j)-postturn_equity[j]), "",
          *holes_by_player_pretty[j])
    
print("\nWinning hand:", getHandClass(np.min(rank_array)))

Flop: 4♠ K♡ K♣
Turn: 6♣
River: T♠

Final hand rankings and improvement vs. turn:
   0%   -8%  7♡ 4♡
   0%  -12%  6♡ 7♣
   0%  -54%  6♠ J♣
   0%   -8%  A♡ 8♡
 100%  +86%  T♢ A♢
   0%   -3%  5♣ J♢

Winning hand: Two pair
